# Programming Assignment 3

Character-level Language Modeling

In [1]:
import numpy as np

In [2]:
with open('shakespeare.txt', 'r') as f:
    text = f.read()

text = text.lower()

In [3]:
Tx = 40
stride = 3
inputs = []
targets = []
for i in range(0, len(text) - Tx, stride):
    inputs.append(text[i:i+Tx])
    targets.append(text[i+1:i+Tx+1])

In [4]:
chars = set(text)
vocab_size = len(chars)
char_to_ix = {ch: i for i, ch in enumerate(sorted(chars))}
ix_to_char = {i: ch for i, ch in enumerate(sorted(chars))}

In [5]:
def to_one_hot(char):
    one_hot = np.zeros((vocab_size,))
    one_hot[char_to_ix[char]] = 1
    return one_hot

In [6]:
X = np.zeros((len(inputs), Tx, vocab_size))
Y = np.zeros((len(inputs), Tx, vocab_size))
for i, (input_seq, target_seq) in enumerate(zip(inputs, targets)):
    for j, char in enumerate(input_seq):
        X[i, j, :] = to_one_hot(char)
    for j, char in enumerate(target_seq):
        Y[i, j, :] = to_one_hot(char)

print(f"Number of train samples: {X.shape[0]}")
print(f"Size of the vocabulary: {vocab_size}")

Number of train samples: 31213
Size of the vocabulary: 38


In [16]:
X

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0.

In [17]:
Y

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [24]:
print(len(X))
print(len(Y))

31213
31213


In [ ]:
import tensorflow as tf


Tx = 40
Ty = 40
n_a = 128  
n_layers = 2  
learning_rate = 0.01
batch_size = 128
num_epochs = 1000


model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(n_a, return_sequences=True, input_shape=(Tx, len(chars))),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(n_a, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(chars), activation='softmax')
])


model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy', metrics=['accuracy'])


history = model.fit(X, Y, batch_size=batch_size, epochs=num_epochs)
model.save('shakespeare_model.h5')

Epoch 1/1000
244/244 [==============================] - 142s 548ms/step - loss: 2.7211 - accuracy: 0.2342
Epoch 2/1000
244/244 [==============================] - 140s 574ms/step - loss: 1.8633 - accuracy: 0.4415
Epoch 3/1000
244/244 [==============================] - 143s 588ms/step - loss: 1.7125 - accuracy: 0.4795
Epoch 4/1000
244/244 [==============================] - 144s 588ms/step - loss: 1.6413 - accuracy: 0.4974
Epoch 5/1000
244/244 [==============================] - 147s 602ms/step - loss: 1.5961 - accuracy: 0.5083
Epoch 6/1000
244/244 [==============================] - 142s 580ms/step - loss: 1.5636 - accuracy: 0.5171
Epoch 7/1000
244/244 [==============================] - 142s 580ms/step - loss: 1.5401 - accuracy: 0.5223
Epoch 8/1000
244/244 [==============================] - 145s 596ms/step - loss: 1.5216 - accuracy: 0.5269
Epoch 9/1000
244/244 [==============================] - 144s 589ms/step - loss: 1.5069 - accuracy: 0.5315
Epoch 10/1000
244/244 [=======================

244/244 [==============================] - 142s 583ms/step - loss: 1.3229 - accuracy: 0.5806
Epoch 78/1000
244/244 [==============================] - 143s 588ms/step - loss: 1.3215 - accuracy: 0.5821
Epoch 79/1000
244/244 [==============================] - 142s 582ms/step - loss: 1.3209 - accuracy: 0.5817
Epoch 80/1000
244/244 [==============================] - 143s 587ms/step - loss: 1.3194 - accuracy: 0.5819
Epoch 81/1000
244/244 [==============================] - 145s 592ms/step - loss: 1.3197 - accuracy: 0.5824
Epoch 82/1000
244/244 [==============================] - 142s 582ms/step - loss: 1.3182 - accuracy: 0.5826
Epoch 83/1000
244/244 [==============================] - 143s 587ms/step - loss: 1.3180 - accuracy: 0.5825
Epoch 84/1000
244/244 [==============================] - 142s 583ms/step - loss: 1.3196 - accuracy: 0.5818
Epoch 85/1000
244/244 [==============================] - 42069s 173s/step - loss: 1.3160 - accuracy: 0.5832
Epoch 86/1000
244/244 [===========================

244/244 [==============================] - 145s 595ms/step - loss: 1.2779 - accuracy: 0.5950
Epoch 154/1000
244/244 [==============================] - 143s 586ms/step - loss: 1.2774 - accuracy: 0.5948
Epoch 155/1000
244/244 [==============================] - 147s 604ms/step - loss: 1.2768 - accuracy: 0.5953
Epoch 156/1000
244/244 [==============================] - 148s 606ms/step - loss: 1.2767 - accuracy: 0.5962
Epoch 157/1000
244/244 [==============================] - 144s 592ms/step - loss: 1.2771 - accuracy: 0.5954
Epoch 158/1000
244/244 [==============================] - 145s 595ms/step - loss: 1.2721 - accuracy: 0.5965
Epoch 159/1000
244/244 [==============================] - 146s 597ms/step - loss: 1.2755 - accuracy: 0.5959
Epoch 160/1000
244/244 [==============================] - 149s 609ms/step - loss: 1.2744 - accuracy: 0.5954
Epoch 161/1000
244/244 [==============================] - 153s 625ms/step - loss: 1.2737 - accuracy: 0.5961
Epoch 162/1000
244/244 [===================

244/244 [==============================] - 190s 781ms/step - loss: 1.2523 - accuracy: 0.6034
Epoch 230/1000
244/244 [==============================] - 194s 797ms/step - loss: 1.2513 - accuracy: 0.6025
Epoch 231/1000
244/244 [==============================] - 180s 736ms/step - loss: 1.2521 - accuracy: 0.6027
Epoch 232/1000
244/244 [==============================] - 165s 675ms/step - loss: 1.2517 - accuracy: 0.6038
Epoch 233/1000
244/244 [==============================] - 154s 632ms/step - loss: 1.2523 - accuracy: 0.6033
Epoch 234/1000
244/244 [==============================] - 148s 606ms/step - loss: 1.2519 - accuracy: 0.6026
Epoch 235/1000
244/244 [==============================] - 165s 676ms/step - loss: 1.2506 - accuracy: 0.6033
Epoch 236/1000
244/244 [==============================] - 173s 709ms/step - loss: 1.2495 - accuracy: 0.6035
Epoch 237/1000
244/244 [==============================] - 172s 705ms/step - loss: 1.2473 - accuracy: 0.6044
Epoch 238/1000
244/244 [===================

Epoch 305/1000
244/244 [==============================] - 197s 806ms/step - loss: 1.2347 - accuracy: 0.6091
Epoch 306/1000
244/244 [==============================] - 161s 660ms/step - loss: 1.2347 - accuracy: 0.6086
Epoch 307/1000
244/244 [==============================] - 186s 764ms/step - loss: 1.2349 - accuracy: 0.6083
Epoch 308/1000
244/244 [==============================] - 154s 632ms/step - loss: 1.2331 - accuracy: 0.6086
Epoch 309/1000
244/244 [==============================] - 154s 632ms/step - loss: 1.2329 - accuracy: 0.6088
Epoch 310/1000
244/244 [==============================] - 151s 620ms/step - loss: 1.2328 - accuracy: 0.6092
Epoch 311/1000
244/244 [==============================] - 173s 708ms/step - loss: 1.2354 - accuracy: 0.6080
Epoch 312/1000
244/244 [==============================] - 190s 777ms/step - loss: 1.2343 - accuracy: 0.6086
Epoch 313/1000
244/244 [==============================] - 198s 811ms/step - loss: 1.2310 - accuracy: 0.6094
Epoch 314/1000
244/244 [====

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Embedding
from tensorflow.keras.models import Model

In [8]:
Tx = 40
vocab_size = 38
embedding_size = 128
hidden_size = 256

input_layer = Input(shape=(Tx, vocab_size))
if embedding_size > 0:
    embedding_layer = Embedding(vocab_size, embedding_size)(input_layer)
    lstm_layer = LSTM(hidden_size, return_sequences=True)(embedding_layer)
else:
    lstm_layer = LSTM(hidden_size, return_sequences=True)(input_layer)
dropout_layer = Dropout(0.2)(lstm_layer)
output_layer = Dense(vocab_size, activation='softmax')(dropout_layer)

model = Model(inputs=input_layer, outputs=output_layer)

ValueError: Input 0 of layer lstm is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 40, 38, 128)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

In [18]:
import tensorflow as tf
import numpy as np

# Load and preprocess the dataset
with open('shakespeare.txt') as f:
    text = f.read()
text = text.lower()  # case folding

# Create character-level vocabulary
chars = sorted(list(set(text)))
vocab_size = len(chars)

# Map characters to indices and vice versa
char_to_index = {char: index for index, char in enumerate(chars)}
index_to_char = {index: char for index, char in enumerate(chars)}

# Create input and target sequences with a fixed window size
Tx = 40  # input sequence length
stride = 3  # stride to move the window
sequences = []  # input sequences
targets = []  # target sequences
for i in range(0, len(text) - Tx, stride):
    sequences.append(text[i:i+Tx])
    targets.append(text[i+1:i+Tx+1])
n_samples = len(sequences)

# Convert sequences and targets to one-hot vectors
X = np.zeros((n_samples, Tx, vocab_size), dtype=np.float32)
Y = np.zeros((n_samples, Tx, vocab_size), dtype=np.float32)
for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        X[i, t, char_to_index[char]] = 1.0
        Y[i, t, char_to_index[targets[i][t]]] = 1.0

# Define the model architecture
embedding_size = 64  # size of the character embedding
hidden_size = 128  # size of the LSTM hidden state
dropout_rate = 0.2  # dropout rate for regularization
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(Tx, vocab_size)),
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_size),
    tf.keras.layers.LSTM(hidden_size, return_sequences=True),
    tf.keras.layers.Dropout(dropout_rate),
    tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(vocab_size, activation='softmax'))
])

# Compile the model
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.CategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss)

# Train the model
batch_size = 128
epochs = 1000
history = model.fit(X, Y, batch_size=batch_size, epochs=epochs)

# Generate text with the trained model
seed_text = 'shall i compare thee to a summer\'s day?\n'
generated_text = seed_text
for i in range(1000):
    x = np.zeros((1, Tx, vocab_size), dtype=np.float32)
    for t, char in enumerate(generated_text[-Tx:]):
        x[0, t, char_to_index[char]] = 1.0
    y = model.predict(x)[0][-1]
    next_char_index = np.argmax(y)
    next_char = index_to_char[next_char_index]
    generated_text += next_char

print(generated_text)


ValueError: Input 0 of layer lstm_4 is incompatible with the layer: expected ndim=3, found ndim=4. Full shape received: (None, 40, 38, 64)

In [19]:
import numpy as np

# read the file and convert all characters to lowercase
with open("shakespeare.txt", "r") as f:
    text = f.read().lower()

# create a set of unique characters in the text
chars = sorted(list(set(text)))

# create a dictionary that maps each character to a unique integer
char_to_int = {c: i for i, c in enumerate(chars)}

# create a dictionary that maps each unique integer back to its corresponding character
int_to_char = {i: c for i, c in enumerate(chars)}

# define the input and output sequence lengths
Tx = 40
Ty = 40

# create input and output sequences using a sliding window approach
X = []
Y = []
for i in range(0, len(text) - Tx, 3):
    X.append([char_to_int[c] for c in text[i:i+Tx]])
    Y.append([char_to_int[c] for c in text[i+1:i+1+Ty]])

# convert X and Y to numpy arrays
X = np.array(X)
Y = np.array(Y)

# convert each character to a one-hot vector
X = np.eye(len(chars))[X]
Y = np.eye(len(chars))[Y]

# print the number of train samples and the size of the vocabulary
print("Number of train samples:", X.shape[0])
print("Size of vocabulary:", len(chars))


Number of train samples: 31213
Size of vocabulary: 38


In [20]:
X

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0.

In [21]:
Y

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [ ]:
Tx = 40
vocab_size = 38
embedding_size = 128
hidden_size = 256


In [23]:
import tensorflow as tf

vocab_size = 38
embedding_dim = 128
rnn_units = 4
learning_rate = 0.001
batch_size = 20
num_epochs = 1000

def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    return model


model = build_model(vocab_size, embedding_dim, rnn_units)


optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer=optimizer, loss=loss_fn)

history = model.fit(X, Y, batch_size=batch_size, epochs=num_epochs)


Epoch 1/1000


ValueError: in user code:

    C:\Users\Pinky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:805 train_function  *
        return step_function(self, iterator)
    C:\Users\Pinky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Pinky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Pinky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Pinky\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\Users\Pinky